In [88]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [89]:
theater_seat_organizer = gp.Model()

# The Parameters

In [90]:
# Uncomment the single file that will be tested and recomment the previous case

test_case = "testcase1-poly.txt"
#test_case = "testcase2-poly.txt"
#test_case = "testcase3-poly.txt"
#test_case = "testcase4-poly.txt"

f = open(test_case,"r")
testfile = f.readlines()

test_name = testfile[0].split("=")[1]

string_num_of_customers_in_group = testfile[1].split(" = ")[1]

#Convert list of string to list of int
number_of_customers_in_group_list = string_num_of_customers_in_group.strip("[").strip("\n").strip("]").split(",")
map_list = map(int, number_of_customers_in_group_list)
number_of_customers_in_group = list(map_list)

number_of_seats = 12 #int(testfile[2].split(" = ")[1])

group_bonus = int(testfile[3].split(" = ")[1])


priority_mappings = []

for group_num in range(len(number_of_customers_in_group)):
    priority_entry = testfile[4+group_num].strip("[").strip("\n").strip("]").split(",")
    map_list = map(int, priority_entry)
    priority_mappings.append(list(map_list))
    map_list = []

print(priority_mappings)


[[0, 1, 0], [1, 0, 0], [0, 0, 0]]


# Decision Variables

In [91]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")




# The Model

In [92]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    for group_num_2 in range(len(number_of_customers_in_group)):
        objectiveFunction += group_bonus*0.5*priority_mappings[group_num][group_num_2]*groupAccepted[group_num]*groupAccepted[group_num_2]

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [93]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [94]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [95]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

In [96]:
# the whole group must be added
for group_num in range(len(number_of_customers_in_group)):
    
    group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
    is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not
    
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   
    
    theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")



# Optimize 

In [97]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1415 rows, 171 columns and 3279 nonzeros
Model fingerprint: 0xa58f4a5d
Model has 1 quadratic objective term
Variable types: 0 continuous, 171 integer (171 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1271 rows and 0 columns
Presolve time: 0.01s
Presolved: 145 rows, 172 columns, 1715 nonzeros
Variable types: 0 continuous, 172 integer (172 binary)

Root relaxation: objective 1.700000e+01, 130 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.00000    0   56   -0.00000   17.000

# Post Process

In [98]:
seating_scheme = list()
for seat in range(number_of_seats):
    seatFound = False
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            result = person_inGroup_atSeat[customer_index, group_num, seat]
            
            if( result.x == 1 ):
                seatFound = True
                print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))
                
                seating_scheme.append((customer_index, group_num))
    
    if(not seatFound):
        print("Seat" + str(seat) + ": <empty>")
        seating_scheme.append(None)

Seat0: person4 fromGroup1
Seat1: person0 fromGroup1
Seat2: <empty>
Seat3: person1 fromGroup0
Seat4: <empty>
Seat5: person0 fromGroup0
Seat6: <empty>
Seat7: person2 fromGroup0
Seat8: <empty>
Seat9: person2 fromGroup1
Seat10: person3 fromGroup1
Seat11: person1 fromGroup1


In [99]:
genericSpacing = list()
last_group = -1
for value in seating_scheme:
    if(type(value) == tuple and value[1] != last_group):
        genericSpacing.append((None,None))
        genericSpacing.append(value)
        last_group = value[1]
        
    
    elif(type(value) == tuple and value[1] == last_group):
        genericSpacing.append(value)
        
genericSpacing.pop(0)

while(genericSpacing[0][1] == genericSpacing[-1][1]):
    genericSpacing.append(genericSpacing[0])
    genericSpacing.pop(0)
    
genericSpacing.pop(0)
    
for members in genericSpacing:
    if not (members[0] == None):
        print("person " + str(members[0] + 1) + " from group " + str(members[1] + 1))
    else:
        print("<SPACE>")



person 2 from group 1
person 1 from group 1
person 3 from group 1
<SPACE>
person 3 from group 2
person 4 from group 2
person 2 from group 2
person 5 from group 2
person 1 from group 2


In [100]:
for group_num in range(len(groupAccepted)):
    if(groupAccepted[group_num].x != 1):
        print("Group " + str(group_num + 1) + " is excluded")
    else:
        print("Group " + str(group_num + 1) + " is accepted")
#print(groupAccepted)

Group 1 is accepted
Group 2 is accepted
Group 3 is excluded
